# Importera de externa paketen


Kör du koden för första gången måste du köra följande tre pip install kommandon och sedan starta trycka på restart Kernel. Börjar koden plötsligt att funka kan det vara värt att köra detta igen. 

In [117]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [118]:
pip install styleframe

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [119]:
pip install xlsxwriter

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [120]:
import pandas as pd
import numpy as np
import datamodels as dm
import openpyxl as op
import datetime
import time
import datetime
import math
import glob
import os 

from datamodels import DimensionReader
from datamodels import NewPearlReader
from styleframe import StyleFrame

# Skapa databasläsarna

In [121]:
scd = DimensionReader()
p = dm.NewPearlReader()

# Definera datum

In [122]:
todaysDate = datetime.datetime.now()
#todaysDate = datetime.datetime(2022, 6, 8)
#manualCalculationDate = datetime.datetime(2022, 6, 7)


Flytta # tecknet till "todaysDate = datetime.datetime(2022, 4, 14)" och ta bort det från "todaysDate = date.today()" för att köra dagens datum.
Ta bort # tecknet om man vill hård sätta beräknat värde till ett visst datum och inte använda logiken för att ta fram senast bankdagen

# Simcorp: Läs in svenska irreguljära icke bankdagar 

In [123]:
query = """
SELECT IRREGULAR_NONBANKING_DAYS FROM SCDAT.A_HOLIDAYS
WHERE CALENDAR = 'SE'
"""

In [124]:
dfLimitik = scd.read(
  query
  )

# Ta fram senaste bankdagen

In [125]:
offset = max(1, (todaysDate.weekday() + 6) % 7 - 3)
timedelta = datetime.timedelta(offset)
lastWeekdayDate = todaysDate - timedelta

for index in dfLimitik.irregular_nonbanking_days: 
    if(pd.to_datetime(index).date() == lastWeekdayDate.date()):
        lastWeekdayDate = pd.to_datetime(lastWeekdayDate)-datetime.timedelta(days=1)
        if lastWeekdayDate.weekday() > 4:
            offset = max(1, (lastWeekdayDate.weekday() + 6) % 7 - 3)
            timedelta = datetime.timedelta(offset)
            lastWeekdayDate = lastWeekdayDate - timedelta
            for index in dflimitik.irregular_nonbanking_days: 
                if(pd.to_datetime(index).date() == lastWeekdayDate.date()):
                    lastWeekdayDate = pd.to_datetime(lastWeekdayDate)-datetime.timedelta(days=1)
                    for index in dflimitik.irregular_nonbanking_days: 
                        if(pd.to_datetime(index).date() == lastWeekdayDate.date()):
                            lastReportedDate = pd.to_datetime(lastWeekdayDate)-datetime.timedelta(days=1)

In [126]:
if 'manualCalculationDate' in locals():
    lastWeekdayDate = manualCalculationDate

# Konvertera datumen till olika format 

In [127]:
todaysDateWithLine = todaysDate.strftime('%Y-%m-%d')
todaysDateNoLine = todaysDate.strftime('%Y%m%d')
lastWeekdayDateWithLine = lastWeekdayDate.strftime('%Y-%m-%d')
lastWeekdayDateNoLine = lastWeekdayDate.strftime('%Y%m%d')
lastWeekdayDateWithLineUS =  lastWeekdayDate.strftime('%d-%m-%Y')

# Kontroll datum 

In [128]:
print("Idag är: " + str(todaysDateWithLine) + ", Beräkningen är för: " + str(lastWeekdayDateWithLine))

Idag är: 2022-06-13, Beräkningen är för: 2022-06-10


Om ovan av någon anlending är fel så använd variabeln manualCalculationDate några rutor upp

# Definera input filer

In [129]:
fileshare = '/home/jovyan/shared/Kort Netto Beräkning/'
lastReportedDatefileAtAll= '/home/jovyan/shared/Kort Netto Beräkning/Reported Values/ReportedPositions_' + '*'


In [130]:
#Hämtar senaste filen för en viss dag. Reported File är alltid senast skapade filen 
list_of_files = glob.glob(lastReportedDatefileAtAll) 
ReportedPositionsExcel = max(list_of_files, key=os.path.getctime)

# Kontroll datum 

In [131]:
print("Idag är: " + str(todaysDateWithLine) + ", Beräkningen är för: " + str(lastWeekdayDateWithLine))
print("Använda ReportedPositionsExcel: " + ReportedPositionsExcel)

Idag är: 2022-06-13, Beräkningen är för: 2022-06-10
Använda ReportedPositionsExcel: /home/jovyan/shared/Kort Netto Beräkning/Reported Values/ReportedPositions_20220610213315.xlsx


In [132]:
OMXS30_Static_Names = fileshare + 'StaticFiles/OMXS30_Static_Names.xlsx'
UBSFile= fileshare + 'Filetransfer/FileUBS/UBS_AMC_Valuation_QQUBCSAPAQLS_' + lastWeekdayDateWithLineUS +'.xls'
OMXS30_Nasdaq_File_CSV = fileshare + 'Filetransfer/FileNasdaqCSV/OMXS30_EOD_'+ lastWeekdayDateNoLine + '.csv'

#Avkommentera om en specefik fil behöver anges 
#OMXS30_Nasdaq_File = fileshare + 'Filetransfer/FileNasdaq/EODWeightings_20220509_OMXS30.xlsx'
#UBSFIL = fileshare + 'Filetransfer/FileUBS/UBS_AMC_Valuation_QQUBCSAPAQLS_09-05-2022.xls'
#ReportedPositionsExcel = fileshare + 'Reported Values/ReportedPositions_20220509.xlsx'

print(OMXS30_Static_Names)
print(OMXS30_Nasdaq_File_CSV)
print(UBSFile)
print(ReportedPositionsExcel)

/home/jovyan/shared/Kort Netto Beräkning/StaticFiles/OMXS30_Static_Names.xlsx
/home/jovyan/shared/Kort Netto Beräkning/Filetransfer/FileNasdaqCSV/OMXS30_EOD_20220610.csv
/home/jovyan/shared/Kort Netto Beräkning/Filetransfer/FileUBS/UBS_AMC_Valuation_QQUBCSAPAQLS_10-06-2022.xls
/home/jovyan/shared/Kort Netto Beräkning/Reported Values/ReportedPositions_20220610213315.xlsx


Dessa filer styr hela programmet. Om man vill köra med andra än de genererade namnen så använd de som finns kommenterade ovan. ReportedPositionsExcel genererar från körningen dagen innan och är inte nödvändig för att koden ska funka, man kan fortfarande läsa ut resultatet 

In [177]:
list_of_files = glob.glob('/home/jovyan/shared/Kort Netto Beräkning/Reported Values/ReportedPositions_20220603*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)

print(latest_file)

/home/jovyan/shared/Kort Netto Beräkning/Reported Values/ReportedPositions_20220603093613.xlsx


# Skapa klasser


In [134]:
class KortNettoBeräkningsObjekt:
    def __init__(self, symbol, closingPrice, marketValue, indexShares, indexWeight, companyName,isin,
                assetName,PartyNameInSCD,BloombergName,PostionOMXS30,TotalNumberOutstandingSharesPerIssuer,OMX30Nominellt,
                TotalOutstandsingSharesSCB = 0,AvistaInnehav = 0, TRSLongValue = 0,TRSShortWeight = '',
                 TRSShortValue = 0):

        #Från NASDAQ filen
        self.symbol = symbol 
        self.closingPrice = closingPrice
        self.marketValue = marketValue
        self.indexShares = indexShares
        self.indexWeight = indexWeight
        self.companyName = companyName
        self.isin = isin
        self.OMX30Nominellt = OMX30Nominellt
        
        #Från Statiska filen
        self.assetName = assetName
        self.PartyNameInSCD = PartyNameInSCD
        self.BloombergName = BloombergName.split(",")
        self.BloombergNameWithEquity = [s + ' Equity' for s in self.BloombergName]
        
        #Från Simcorp
        self.PostionOMXS30 = PostionOMXS30
        self.TotalOutstandsingSharesSCB = TotalOutstandsingSharesSCB
        self.AvistaInnehav = AvistaInnehav
             
        #Från UBS fil
        self.TRSLongValue = TRSLongValue
        
        #Från Pearl
        self.TRSShortWeight = TRSShortWeight     
        self.TRSShortValue = TRSShortValue                             
            
    def as_dict(self):        
        
        return {'Asset Name (Nasdaq)': self.symbol,
               'Closing Price (Nasdaq)': self.closingPrice,
               'Asset Name (Nasdaq)': self.assetName,
               'ISIN (Nasdaq)': self.isin,
               'Ticker manuellt justerad': listToString(self.BloombergName),
               'Total current number of shares outstanding SCD':self.TotalOutstandsingSharesSCB,
               'Vikt(%)': self.indexWeight,
               "Postion OMXS30":self.PostionOMXS30,
               "Position i bolag via OMX30":self.getPostionInCompany(),
               "Position i bolag via OMX30 Nominellt":self.OMX30Nominellt,
               "Avistainnehav Nominellt": self.AvistaInnehav,
               "Antal aktier - kort nettoposition": self.OMX30Nominellt+self.AvistaInnehav,

                #TRS
               "TRS Långa Värde Nominellt": self.TRSLongValue,
               "TRS Kort Vikt": self.getTRSShortWeight(),
                "TRS Korta Nominellt": self.TRSShortValue,
               "TRS Korta + Långa": self.TRSLongValue+self.TRSShortValue,
               "Totalt netto OMX, avista & TRS":
                
                   self.OMX30Nominellt+
                   self.AvistaInnehav+
                   self.TRSLongValue+
                   self.TRSShortValue
               ,
               "Postion att rapportera":round(self.getPositiontoReport(),4), 
               "Postion att rapportera i %":round(self.getPositiontoReport()*100,4)  
               }
    
    def UpdateTotalOutstandsingSharesSCB(self, NewTotalOutstandsingSharesSCB):
        self.TotalOutstandsingSharesSCB = int(self.TotalOutstandsingSharesSCB)+NewTotalOutstandsingSharesSCB

    def UpdateAvistaInnehav(self, NewAvistaInnehav):
        self.AvistaInnehav = NewAvistaInnehav
        
    def UpdateTRSLongValue(self,NewTRSLongValue):
        self.TRSLongValue = self.TRSLongValue+NewTRSLongValue
        
    def UpdateTRSShortWeight(self, NewTRSShortWeight):
        self.TRSShortWeight = self.TRSShortWeight + ', ' + str(NewTRSShortWeight)     
        
    def UpdateTRSShortValue(self,NewTRSShortValue):
        self.TRSShortValue = self.TRSShortValue + NewTRSShortValue
    
    def getTRSShortWeight(self):
        if self.TRSShortWeight == '':
            return '0'
        else:
            return self.TRSShortWeight[2:]
    
    def getPostionInCompany(self):
        return (self.indexWeight*self.PostionOMXS30)/100 
        
    def getPositiontoReport(self):
        return (((self.OMX30Nominellt+self.AvistaInnehav+self.TRSLongValue+
                 (self.TRSShortValue))/self.TotalOutstandsingSharesSCB)    

Om man någonsin vill ändra någon namn eller göra någon justering på beräkningen är det här den bör ske

In [135]:
#SimCorp klass

class SimCorpSharesOutstanding:
    def __init__(self, SECSHORT, SECNO, SECSERNO, ISIN, ISSVOL, PAR, DATE, BBNAME):

        self.SECSHORT = SECSHORT
        self.SECNO = SECNO
        self.SECSERNO = SECSERNO
        self.ISIN = ISIN
        self.ISSVOL = ISSVOL
        self.PAR = PAR
        self.DATE = DATE
        self.BBNAME = BBNAME

    def as_dict(self):
        return {'Security ID': self.SECSHORT,
                'Security NO.': self.SECNO,
                'Ser. NO': self.SECSERNO,
                'ISIN': self.ISIN,
                'Issued volume': self.ISSVOL,
                'Party': self.PAR,
                'Date': self.DATE,
                'BBName': self.BBNAME
                }

In [136]:
# Hjälp Klass

def listToString(theList):
    # initialize an empty string
    string = ""

    # traverse in the string
    for element in theList:
        string += element + ','

    # return string
    return string


# Läs in Static filen


Vid förändringar av innehav i OMX - alt vid namnändringar - måste denna fil uppdateras

In [137]:
OMXS30StaticNames_DataFrame =  pd.read_excel(OMXS30_Static_Names)
print(OMXS30StaticNames_DataFrame)

   Asset Name Nasdaq                       Asset Name  Party name in SCD  \
0              SKF B                            AB SKF               SKF   
1                ABB                           ABB LTD           ABB LTD   
2               ALFA                     ALFA LAVAL AB        ALFA LAVAL   
3             ASSA B                     ASSA ABLOY AB        ASSA ABLOY   
4                AZN                   ASTRAZENECA PLC       ASTRAZENECA   
5             ATCO B                    ATLAS COPCO AB       ATLAS COPCO   
6             ATCO A                    ATLAS COPCO AB       ATLAS COPCO   
7           ALIV SDB                       AUTOLIV INC           AUTOLIV   
8                BOL                        BOLIDEN AB           BOLIDEN   
9             ELUX B                     ELECTROLUX AB        ELECTROLUX   
10          ESSITY B                         ESSITY AB            ESSITY   
11               EVO         EVOLUTION GAMING GROUP AB  EVOLUTION GAMING   
12          

# Read Simcorp Limit FI-B MAX OMX INDEX to get Postion OMXS30

In [138]:
# Hämar Limitens unika ID
query = """
select LIMITIK from SCDAT.LIMITRULES SCDAT WHERE LIMIT ='FI-B MAX OMX INDEX'
"""

In [139]:
dflimitik = scd.read(
  query
  )

In [140]:
limitik = int(dflimitik.limitik[0])

In [141]:
queryPostionOMXS30 = """select LIMITACT from SCDAT.LIMUTILISATION SCDAT
where LIMITIK = :limitik
and LIMITDATE = TO_DATE(:usedate,'YYYY-MM-DD')
order by LIMITDATE desc"""

In [142]:
dfPostionOMXS30 = scd.read(
  queryPostionOMXS30, params={'usedate': lastWeekdayDateNoLine,'limitik':limitik}
  )
print(dfPostionOMXS30)

       limitact
0 -1.115546e+10


In [143]:
PostionOMXS30 = int(dfPostionOMXS30.limitact[0])
print('PostionOMXS30: ' + str(PostionOMXS30))

PostionOMXS30: -11155463200


# Läs in Dagliga Nasdaq filen

In [144]:
data= pd.read_csv(OMXS30_Nasdaq_File_CSV,skiprows=2,skipfooter= 2, sep=',', engine='python', usecols = ['Symbol',
'Closing Price',
'Market Value',
'Index Shares',
'Index Weight',
'SEDOL',
'Exchange',
'Currency',
'FX Rate',
'Free Float Factor',
'Country Code',
'Industry Code',
'Index Symbol',
'CUSIP',
'Bloomberg ID',
'ISIN',
'Security Shares',
'Capping Factor',
'Security Dividend Market Value',
'ICB Subsector Code'
])

print(data)


      Symbol  Closing Price  Market Value  Index Shares  Index Weight  \
0        ABB         319.40  1.570758e+11  4.917840e+08      3.062085   
1       ALFA         303.80  1.274308e+11  4.194563e+08      2.484177   
2   ALIV SDB         786.00  3.997549e+10  5.085941e+07      0.779295   
3     ASSA B         252.50  2.664002e+11  1.055050e+09      5.193289   
4     ATCO A         479.40  4.024055e+11  8.393941e+08      7.844620   
5     ATCO B         423.50  1.652577e+11  3.902190e+08      3.221586   
6        AZN        1179.80  1.948838e+11  1.651837e+08      3.799126   
7        BOL         434.90  1.189500e+11  2.735112e+08      2.318849   
8     ELUX B         163.45  4.915395e+10  3.007278e+08      0.958223   
9     ERIC B          85.19  2.617374e+11  3.072396e+09      5.102391   
10  ESSITY B         241.00  1.544315e+11  6.407948e+08      3.010537   
11       EVO        1063.00  2.286631e+11  2.151111e+08      4.457630   
12    GETI B         372.10  9.457010e+10  2.541524

# Skapar objekt

In [180]:
#skapar tom lista
OMX30List = []

In [181]:
for row in data.iterrows():
    
    assetName = OMXS30StaticNames_DataFrame['Asset Name '].where(OMXS30StaticNames_DataFrame['Asset Name Nasdaq'] == row[1].Symbol).dropna().values[0]
    PartyNameInSCD = OMXS30StaticNames_DataFrame['Party name in SCD'].where(OMXS30StaticNames_DataFrame['Asset Name Nasdaq'] == row[1].Symbol).dropna().values[0]
    BloombergNamn = OMXS30StaticNames_DataFrame['Bloomberg namn'].where(OMXS30StaticNames_DataFrame['Asset Name Nasdaq'] == row[1].Symbol).dropna().values[0]
    
    #Alla förekomster av " SW " i BloombergNamn från Static filen konverteras till " SE " för att hitta rätt namn i
    fixedSimCorpBBName = BloombergNamn.replace(' SW ',' SE ')
       
    TotalNumberOutstandingSharesPerIssuer= ""
    print('assetName: ' + assetName)
    print('PartyNameInSCD: ' + PartyNameInSCD)
    print('BloombergNamn: ' + fixedSimCorpBBName)
    
    symbol = row[1]['Symbol']
    closingPrice = row[1]['Closing Price']
    marketValue = row[1]['Market Value']
    indexShares = row[1]['Index Shares']
    companyName = row[1]['SEDOL']
    isin = row[1]['ISIN']
    
    indexWeight = row[1]['Index Weight']
    postionCompany = (indexWeight*PostionOMXS30)/100
    OMX30Nominellt =(postionCompany/closingPrice)
    
    for värde in OMX30List:
            if(värde.assetName == assetName):
                indexWeight = indexWeight +värde.indexWeight
                print("Värde med samma assetname = " + assetName + " .För över värden till det senaste värdet")
                print("Nuvarande vikt: " + str(värde.indexWeight))
                print("Ny vikt: " + str(indexWeight))
                print("Nuvarande nominellt värde: " + str(värde.OMX30Nominellt))
                print("Nytt nominellt värde: " + str(OMX30Nominellt))
                print("Row with name " + str(värde.symbol) + " will be removed")
                OMX30Nominellt = OMX30Nominellt + värde.OMX30Nominellt 
                OMX30List.remove(värde)

    tempStorageKortNettoBeräkningsObjekt = KortNettoBeräkningsObjekt(
        symbol,
        closingPrice,
        marketValue,
        indexShares,
        indexWeight,
        companyName,
        isin,
        assetName,
        PartyNameInSCD,
        fixedSimCorpBBName,
        PostionOMXS30,
        TotalNumberOutstandingSharesPerIssuer,
        OMX30Nominellt
    )
    OMX30List.append(tempStorageKortNettoBeräkningsObjekt)


assetName: ABB LTD
PartyNameInSCD: ABB LTD
BloombergNamn: ABBN SE
assetName: ALFA LAVAL AB
PartyNameInSCD: ALFA LAVAL
BloombergNamn: ALFA SS
assetName: AUTOLIV INC
PartyNameInSCD: AUTOLIV
BloombergNamn: ALIV SS
assetName: ASSA ABLOY AB
PartyNameInSCD: ASSA ABLOY
BloombergNamn: ASSAB SS
assetName: ATLAS COPCO AB
PartyNameInSCD: ATLAS COPCO
BloombergNamn: ATCOA SS,ATCOB SS
assetName: ATLAS COPCO AB
PartyNameInSCD: ATLAS COPCO
BloombergNamn: ATCOA SS,ATCOB SS
Värde med samma assetname = ATLAS COPCO AB .För över värden till det senaste värdet
Nuvarande vikt: 7.8446196
Ny vikt: 11.06620601
Nuvarande nominellt värde: -1825414.3776762355
Nytt nominellt värde: -848601.8569628126
Row with name ATCO A will be removed
assetName: ASTRAZENECA PLC
PartyNameInSCD: ASTRAZENECA
BloombergNamn: AZN SS,AZN LN
assetName: BOLIDEN AB
PartyNameInSCD: BOLIDEN
BloombergNamn: BOL SS
assetName: ELECTROLUX AB
PartyNameInSCD: ELECTROLUX
BloombergNamn: ELUXB SS,ELUXA SS
assetName: TELEFONAKTIEBOLAGET LM ERICSSON
Par

# Hämta Outstanding Securities

In [147]:
#Konverar samtliga instrument i 'Party name in SCD' för att kunna använda dom i SQL frågan 
joined_string = "'"+ "','".join(OMXS30StaticNames_DataFrame['Party name in SCD']) + "'"

In [148]:
queryPARIK = """
select PARIK FROM scdat.PARTNERS WHERE par In (%s)
""" % joined_string

In [149]:
# Hämtar ID för Issuer i DB
dfPARIK = scd.read(
  queryPARIK,params={'joined_string': joined_string}
  )

In [150]:
#Konverar resultatet från frågan ovan för att kunna använda dom i nästa SQL frågan 
converted_list = [str(element) for element in dfPARIK['parik']]
joined_string_parik = "'"+ "','".join(converted_list) + "'"

In [151]:
#hämtar No. ooutstanding shares fr DB

query = """SELECT A.SECSHORT, A.SECNO, A.SECSERNO, A.ISIN, C.SHAREISSVOL, B.PAR, C.SHAREDATE, A.BLOOMBERGCODE, A.SECNAME
FROM SCDAT.SECURITIES A
join SCDAT.PARTNERS B on a.PARIK = B.PARIK
JOIN SCDAT.SHARECAPITAL C ON A.SECIK = C.SECIK
WHERE A.SECTYPEIK in (SELECT SECTYPEIK FROM SCDAT.SECTYPES WHERE SECTYPE = 'EQUITY')
and A.hidden = 0 
AND A.PARIK in (%s)
AND SHAREDATE =TO_DATE(:usedate,'YYYY-MM-DD')
ORDER BY A.SECNO""" % (joined_string_parik)

In [152]:
readLimitik = scd.read(
    query,params={'limitik':limitik,'usedate': lastWeekdayDateNoLine}
  )

In [153]:
SimcorpTempStorage = 0
SimcorpList = []

for i,row in readLimitik.iterrows(): 
    print(row[0])
    print(row[1])
    print(row[2])
    print(row[3])
    print(row[4])
    print(row[5])
    print(row[6]) 
    print(row[7])
    tempStorageSimCorpSharesOutstanding = SimCorpSharesOutstanding(row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7])
    SimcorpList.append(tempStorageSimCorpSharesOutstanding)

ABBN SW
1223157
0
CH0012221716
1581523060.0
ABB LTD
2022-06-10 00:00:00
ABBN SW Equity
BOL SS
1227004
0
SE0017768716
273511169.0
BOLIDEN
2022-06-10 00:00:00
BOL SS Equity
HEXAB SS
1227035
0
SE0015961909
2595227888.0
HEXAGON
2022-06-10 00:00:00
HEXAB SS Equity
KINVA SS
1227124
0
SE0015810239
33755432.0
KINNEVIK
2022-06-10 00:00:00
KINVA SS Equity
KINVB SS
1227125
0
SE0015810247
242418116.0
KINNEVIK
2022-06-10 00:00:00
KINVB SS Equity
ALFA SS
1227217
0
SE0000695876
413876823.0
ALFA LAVAL
2022-06-10 00:00:00
ALFA SS Equity
ALV UN
600056
0
US0528001094
55290758.0
AUTOLIV
2022-06-10 00:00:00
ALV UN Equity
ABB SS
600156
0
CH0012221716
471625204.0
ABB LTD
2022-06-10 00:00:00
ABB SS Equity
AZN SS
600160
0
GB0009895292
1549109281.0
ASTRAZENECA
2022-06-10 00:00:00
AZN SS Equity
ASSAB SS
600164
0
SE0007100581
1055050365.0
ASSA ABLOY
2022-06-10 00:00:00
ASSAB SS Equity
ATCOA SS
600166
0
SE0017486889
3357576384.0
ATLAS COPCO
2022-06-10 00:00:00
ATCOA SS Equity
ATCOB SS
600167
0
SE0017486897
1560876

In [154]:
for index in OMX30List:
        for SimcorpTempStorage in SimcorpList:
            if(index.PartyNameInSCD == SimcorpTempStorage.PAR):
                index.UpdateTotalOutstandsingSharesSCB(SimcorpTempStorage.ISSVOL) 
                print(index.PartyNameInSCD + ' Add ' + str(SimcorpTempStorage.ISSVOL))

ABB LTD Add 1581523060.0
ABB LTD Add 471625204.0
ALFA LAVAL Add 413876823.0
AUTOLIV Add 55290758.0
AUTOLIV Add 46984544.0
ASSA ABLOY Add 1055050365.0
ASSA ABLOY Add 57525969.0
ATLAS COPCO Add 3357576384.0
ATLAS COPCO Add 1560876032.0
ASTRAZENECA Add 1549109281.0
ASTRAZENECA Add 361331.0
BOLIDEN Add 273511169.0
ELECTROLUX Add 274885045.0
ELECTROLUX Add 8192348.0
ERICSSON Add 261755983.0
ERICSSON Add 3072395752.0
ESSITY Add 641053575.0
ESSITY Add 61288914.0
EVOLUTION GAMING Add 215111115.0
GETINGE IND Add 254152373.0
GETINGE IND Add 18217200.0
HEXAGON Add 2595227888.0
HEXAGON Add 110250000.0
H&M Add 1460672000.0
H&M Add 194400000.0
INVESTOR Add 1821936744.0
INVESTOR Add 1246763376.0
KINNEVIK Add 33755432.0
KINNEVIK Add 242418116.0
KINNEVIK Add 2788199.0
NORDEA BANK Add 3789514793.0
SANDVIK Add 1254385923.0
SCA Add 64581994.0
SCA Add 637760495.0
SEB Add 2154569426.0
SEB Add 24152508.0
SHB Add 1944777165.0
SHB Add 35251329.0
SINCH AB Add 837380778.0
SKANSKA Add 19660647.0
SKANSKA Add 40024

# Hämta AvistaInnehav

In [155]:
query ="""SELECT L.LIMGROUPVAL, L.LIMITACTGRPVAL, U.LIMIT_REPORT_DATE 
FROM SCDAT.LIMUTILCOMPLDETAIL L, SCDAT.A_LIMITS N, SCDAT.A_LIMITS_UTILISATION U
where
N.LIMIT= 'OMX30 - BALANCE NOM'
and N.LIMIT_REF=U.LIMIT_REF
and U.LIMIT_REPORT_DATE =TO_DATE(:usedate,'YYYY-MM-DD')
and U.LIMIT_UTILISATION_REF=L.LIMUTILIK"""

In [156]:
AvistaRead = scd.read(
    query,params={'usedate': lastWeekdayDateNoLine}
  )

In [157]:
for value in AvistaRead.iterrows():
    print(value)

(0, limgroupval                      ABB LTD
limitactgrpval                  350571.0
limit_report_date    2022-06-10 00:00:00
Name: 0, dtype: object)
(1, limgroupval                   ALFA LAVAL
limitactgrpval                 1344932.0
limit_report_date    2022-06-10 00:00:00
Name: 1, dtype: object)
(2, limgroupval                   ASSA ABLOY
limitactgrpval                 5222338.0
limit_report_date    2022-06-10 00:00:00
Name: 2, dtype: object)
(3, limgroupval                  ASTRAZENECA
limitactgrpval                 1597191.0
limit_report_date    2022-06-10 00:00:00
Name: 3, dtype: object)
(4, limgroupval                  ATLAS COPCO
limitactgrpval                41252968.0
limit_report_date    2022-06-10 00:00:00
Name: 4, dtype: object)
(5, limgroupval                      AUTOLIV
limitactgrpval                  739491.0
limit_report_date    2022-06-10 00:00:00
Name: 5, dtype: object)
(6, limgroupval                      BOLIDEN
limitactgrpval                  107733.0
limit_re

In [158]:
addedItems = []

for index in OMX30List:
    for value in AvistaRead.iterrows():
        if(index.PartyNameInSCD == value[1].limgroupval and index.PartyNameInSCD not in addedItems):
            print(value[1].limgroupval)
            print(value[1].limitactgrpval)
            addedItems.append(index.PartyNameInSCD)
            index.UpdateAvistaInnehav(value[1].limitactgrpval)

ABB LTD
350571.0
ALFA LAVAL
1344932.0
AUTOLIV
739491.0
ASSA ABLOY
5222338.0
ATLAS COPCO
41252968.0
ASTRAZENECA
1597191.0
BOLIDEN
107733.0
ELECTROLUX
780.0
ERICSSON
186649.0
ESSITY
37478.0
GETINGE IND
20385950.0
HEXAGON
3482213.0
H&M
22363258.0
INVESTOR
3682.0
NORDEA BANK
150000.0
SANDVIK
12048693.0
SCA
310851.0
SEB
20867329.0
SHB
24960.0
SINCH AB
32506590.0
SKANSKA
2557076.0
SWEDBANK
9146.0
TELE2
71723.0
TELIASONERA
222292.0
VOLVO
14519825.0


# Hämta TRS Långa

In [159]:
UBS_DataFrame =  pd.read_excel(UBSFile, sheet_name = 'Equity',  skiprows=21)

In [160]:
for index in OMX30List:
    for namn in index.BloombergName:
        if(namn == 'ATCOA SS'):
            namn = 'ATCO A SS'
        if(namn == 'ATCOB SS'):
            namn = 'ATCO B SS'
        #print(namn)
        for row in UBS_DataFrame.iterrows():
            #print(row[1].BBG)
            if namn == row[1].BBG:
                index.UpdateTRSLongValue(row[1].Shares)
                print('Updated: ' + namn)
                print(row[1].Shares)


Updated: ASSAB SS
63405.54
Updated: ATCO A SS
235562.355
Updated: EVO SS
10011.42
Updated: HEXAB SS
26500.77
Updated: HMB SS
77146.695
Updated: INVEB SS
66742.65


# GDDUWI and USD/SEK


In [161]:
UBS_DataFrame =  pd.read_excel(UBSFile, sheet_name = 'Index', skiprows=21)
usdGDDUWI = UBS_DataFrame['Total Val. (USD)'].iloc[0]
print('USD GDDUWI ' + str(usdGDDUWI))

USD GDDUWI -434003033.90997


In [162]:
queryfxrateintqp = """select  from_ccy_id,
        to_ccy_id,
        vdate,
        value
from    out_exrate
where   vdate = TO_DATE(:usedate,'YYYY/MM/DD')
    and fxset_id = 'D'
    and from_ccy_id not like '%0'
    and from_CCY_ID = 'USD'
"""

In [163]:
dffxrateintqp = p.read(
  queryfxrateintqp,params={'usedate': lastWeekdayDateNoLine}
  )
print(dffxrateintqp)

  from_ccy_id to_ccy_id      vdate     value
0         USD       SEK 2022-06-10  10.00855


In [164]:
exchagnerate = dffxrateintqp.value[0]
print('Exchagnerate ' + str(exchagnerate))

Exchagnerate 10.00855


In [165]:
sekGDDUWI = usdGDDUWI*exchagnerate
print('SED GDDUWI ' + str(sekGDDUWI))

SED GDDUWI -4343741065.03963


# Hämta TRS från inläst Bloomberg fil

In [166]:
query = '''select
           calculation_date,
            nvl(isin, 'N/A') isinvalue,
            security_name,  
            nr_of_shares_today_index*foreign_inclusion_fctr*REPLACE(reserved_5, '.', ',') test
             ,
            NR_OF_SHARES_ND_INDEX * foreign_inclusion_fctr * REPLACE(reserved_5, '.', ',') * price * fx.value mv_base
            ,
            NR_OF_SHARES_ND_INDEX * foreign_inclusion_fctr * REPLACE(reserved_5, '.', ',') * price * fx.value
            / sum(NR_OF_SHARES_ND_INDEX * foreign_inclusion_fctr * REPLACE(reserved_5, '.', ',') * price * fx.value) over() weight,
            e.*


from        EXT_BM_MSCI_CONST e
inner join  out_exrate fx
    on      fx.vdate = e.calculation_date
    and     e.ISO_CCY_SYMBOL_OF_PRICE = fx.from_ccy_id   
where       calculation_date = TO_DATE(:usedate,'YYYY-MM-DD')
    and     (developed_markets_flag = 1)
    and     (STANDARD_INF_FLAG = 1 or STD_INF_FLAG_ND = 1)

'''

In [167]:
pearl_data = p.read(query,params={'usedate': lastWeekdayDateNoLine})
print(pearl_data)

     calculation_date     isinvalue          security_name          test  \
0          2022-06-10  AU000000WOW2       WOOLWORTHS GROUP  1.211975e+09   
1          2022-06-10  AU0000224040  WOODSIDE ENERGY GROUP  1.884399e+09   
2          2022-06-10  AU000000APA1              APA GROUP  1.179894e+09   
3          2022-06-10  AU000000SEK6                   SEEK  3.362242e+08   
4          2022-06-10  AU000000DXS1                  DEXUS  1.075565e+09   
...               ...           ...                    ...           ...   
1510       2022-06-10  US6293775085             NRG ENERGY  2.421532e+08   
1511       2022-06-10  US5018892084               LKQ CORP  2.707634e+08   
1512       2022-06-10  GB00B8W67662       LIBERTY GLOBAL A  1.568879e+08   
1513       2022-06-10  US2310211063                CUMMINS  1.424267e+08   
1514       2022-06-10  US12514G1085               CDW CORP  1.349443e+08   

           mv_base    weight index_family calculation_date  \
0     2.935393e+11  0.000

In [168]:
for index in OMX30List:
    for name in index.BloombergNameWithEquity:   
        print(name)

ABBN SE Equity
ALFA SS Equity
ALIV SS Equity
ASSAB SS Equity
ATCOA SS Equity
ATCOB SS Equity
AZN SS Equity
AZN LN Equity
BOL SS Equity
ELUXB SS Equity
ELUXA SS Equity
ERICB SS Equity
ERICA SS Equity
ESSITYB SS Equity
ESSITYA SS Equity
EVO SS Equity
GETIB SS Equity
HEXAB SS Equity
HMB SS Equity
INVEB SS Equity
INVEA SS Equity
KINVB SS Equity
KINVA SS Equity
NDA SS Equity
SAND SS Equity
SCAB SS Equity
SCAA SS Equity
SEBA SS Equity
SHBA SS Equity
SHBB SS Equity
SINCH SS Equity
SKAB SS Equity
SKAA SS Equity
SKFB SS Equity
SKFA SS Equity
SWEDA SS Equity
SWMA SS Equity
TEL2B SS Equity
TEL2A SS Equity
TELIA SS Equity
VOLVB SS Equity
VOLVA SS Equity


In [169]:
queryfxrateintqpVarible = """select  from_ccy_id,
        to_ccy_id,
        vdate,
        value
from    out_exrate
where   
     fxset_id = 'D'
    and from_ccy_id not like '%0'
    and from_CCY_ID = :currency
    AND vdate = TO_DATE(:usedate,'YYYY/MM/DD')
"""

In [170]:
for index in OMX30List:
    for name in index.BloombergNameWithEquity:   
            for value in pearl_data.iterrows():
                #print(value[1].bloomberg_ticker)
                if(value[1].bloomberg_ticker == name):
                   
                    roundWeight = round(value[1].weight,6)
                    index.UpdateTRSShortWeight(roundWeight)
                    AndelExponering = sekGDDUWI*value[1].weight
                    
                    #Hämta valutakurs
                    valutaPearl = value[1].iso_ccy_symbol_of_price      
                    dffxrateintqpCurrency = p.read(queryfxrateintqpVarible,params={'usedate': lastWeekdayDateNoLine,'currency':valutaPearl})
                    kursPearl = dffxrateintqpCurrency.value[0]
                    
                    price = value[1].price
                    valueNominell = AndelExponering/(price*kursPearl)
                    index.UpdateTRSShortValue(valueNominell)
                    
                    print(name)
                    print('Värde nominell ' + str(valueNominell))
                    print('Weight: ' + str(roundWeight))
                    print('AndelExponering: ' + str(AndelExponering))
                    print('Valutan är: ' + str(valutaPearl))
                    print('Andel exponering dividerar med valutakursen ' + str(kursPearl))
                           

ABBN SE Equity
Värde nominell -13989.144615093046
Weight: 0.000939
AndelExponering: -4079848.5719803893
Valutan är: CHF
Andel exponering dividerar med valutakursen 10.126524004654222
ALFA SS Equity
Värde nominell -2500.7232838436134
Weight: 0.000142
AndelExponering: -617178.5064526038
Valutan är: SEK
Andel exponering dividerar med valutakursen 1.0
ASSAB SS Equity
Värde nominell -8536.457017883267
Weight: 0.000461
AndelExponering: -2003506.4620972027
Valutan är: SEK
Andel exponering dividerar med valutakursen 1.0
ATCOA SS Equity
Värde nominell -22876.875013720703
Weight: 0.000556
AndelExponering: -2416255.538949181
Valutan är: SEK
Andel exponering dividerar med valutakursen 1.0
ATCOB SS Equity
Värde nominell -13293.804033019775
Weight: 0.000284
AndelExponering: -1232069.7577802727
Valutan är: SEK
Andel exponering dividerar med valutakursen 1.0
AZN LN Equity
Värde nominell -13196.235560512914
Weight: 0.003697
AndelExponering: -16057838.422443997
Valutan är: GBP
Andel exponering dividerar

OBS: Koden ovan kan ta någon minut att köra igen 

# Skapa output-filnamn


In [171]:
currentTime = datetime.datetime.now()
hours = 2
hours_added = datetime.timedelta(hours = hours)
real_date_and_time = currentTime + hours_added
currentTime = real_date_and_time.strftime('%H%M%S')

filename = fileshare + 'Calculations/Per ' + lastWeekdayDateNoLine +  ' (kontroll)' + todaysDateNoLine + ') ' + str(currentTime)+ '.xlsx'
print('Filnamnet')
print(filename)

reportedPostionFilename = fileshare + 'Reported Values/ReportedPositions_' + lastWeekdayDateNoLine + str(currentTime) + '.xlsx'
print('reportedPostionFilename')
print(reportedPostionFilename)


Filnamnet
/home/jovyan/shared/Kort Netto Beräkning/Calculations/Per 20220610 (kontroll)20220613) 230212.xlsx
reportedPostionFilename
/home/jovyan/shared/Kort Netto Beräkning/Reported Values/ReportedPositions_20220610230212.xlsx


# Skapa Sheets

In [172]:
writer = pd.ExcelWriter(filename)

outputDateFrame = pd.DataFrame([x.as_dict() for x in OMX30List])
outputDateFrame.style.to_excel(writer, sheet_name='Kort Netto Beräknings Objekt', index=False)

workbook = writer.book
worksheet  = writer.sheets['Kort Netto Beräknings Objekt']

#Definerar numberformattet 
a_format = workbook.add_format({'num_format': '#,##0.00'})
a_format.set_align('center')

#Anpassar bredden på kolumen 
for column in outputDateFrame:
    
    column_length = max(outputDateFrame[column].astype(str).map(len).max(), len(column)) 
    col_idx = outputDateFrame.columns.get_loc(column)
    writer.sheets['Kort Netto Beräknings Objekt'].set_column(col_idx, col_idx, column_length,a_format)      

In [173]:
simdimDateFrame = pd.DataFrame(x.as_dict() for x in SimcorpList)
simdimDateFrame.to_excel(writer, sheet_name='No.Shares Outstanding SCD', index=False)

#Anpassar bredden på kolumen 
for column in simdimDateFrame:
    column_length = max(simdimDateFrame[column].astype(str).map(len).max(), len(column))
    col_idx = simdimDateFrame.columns.get_loc(column)
    writer.sheets['No.Shares Outstanding SCD'].set_column(col_idx, col_idx, column_length)

In [174]:
caluclationDataframe = pd.DataFrame()

caluclationDataframe.at[""] = ""
#Lägger till en tom ruta för att få värderna att komma in
caluclationDataframe["GDDUWI i USD"] = usdGDDUWI
caluclationDataframe["Valutakurs"] = exchagnerate
caluclationDataframe["GDDUWI i SEK"] = sekGDDUWI


caluclationDataframe.to_excel(writer, sheet_name='Calculations Data', index=False)

#Anpassar bredden på kolumen 
for column in caluclationDataframe:
    column_length = max(caluclationDataframe[column].astype(str).map(len).max(), len(column))
    col_idx = caluclationDataframe.columns.get_loc(column)
    writer.sheets['Calculations Data'].set_column(col_idx, col_idx, column_length)

print(caluclationDataframe)

  GDDUWI i USD  Valutakurs  GDDUWI i SEK
 -4.340030e+08    10.00855 -4.343741e+09


In [175]:
pearl_data.to_excel(writer, sheet_name='Pearl Data', index=False)

# Skapa den historiska Excelfilen

In [176]:
writer.save()

# Analysera rapportbehovet

In [479]:
for index in OMX30List:
    _postiontoReport = index.getPositiontoReport()*100

    if(_postiontoReport< -0.1):
        print("Beräknad kort nettopositon " + index.symbol + " på nivå " + str(round(_postiontoReport,2)))

    if(index.TotalOutstandsingSharesSCB == 0):
        print("TotalOutstandsingSharesSCB för " + index.symbol + " är 0, kan vara värt att undersöka")
        

Beräknad kort nettopositon BOL på nivå -0.19
Beräknad kort nettopositon ELUX B på nivå -0.24
Beräknad kort nettopositon ERIC B på nivå -0.21
Beräknad kort nettopositon ESSITY B på nivå -0.21
Beräknad kort nettopositon EVO på nivå -0.23
Beräknad kort nettopositon INVE B på nivå -0.13
Beräknad kort nettopositon KINV B på nivå -0.2
Beräknad kort nettopositon SCA B på nivå -0.17
Beräknad kort nettopositon SHB A på nivå -0.23
Beräknad kort nettopositon SKF B på nivå -0.22
Beräknad kort nettopositon SWED A på nivå -0.23
Beräknad kort nettopositon SWMA på nivå -0.24
Beräknad kort nettopositon TEL2 B på nivå -0.21
Beräknad kort nettopositon TELIA på nivå -0.22


In [480]:
class reportValuesObjects:
    def __init__(self, name,reportedDate,previousValue,lowerValue,higherValue,calculatedValue,calculatedValueRound,calculatedDalue):

        self.name = name
        self.previousValue = previousValue
        self.previousValue = previousValue
        self.lowerValue = lowerValue
        self.higherValue = higherValue,
        self.calculatedValue = calculatedValue,
        self.calculatedValueRound = calculatedValueRound
        self.calculatedDalue = calculatedDalue
        
    def hasReportedValue(self):
        return (self.previousValue is not None)


In [481]:
ReportedPositions_DataFrame = pd.read_excel(ReportedPositionsExcel)

#Sorterar bort oanvänd värden 
ReportedPositions_DataFrame['Reported Date'] = pd.to_datetime(ReportedPositions_DataFrame['Reported Date'], dayfirst=True)
previousvalues = ReportedPositions_DataFrame.sort_values('Reported Date').groupby('Asset Name (Nasdaq)').last()
previousvalues = previousvalues.loc[(previousvalues['Reported percent'] < 0.00)]

print('Senaste rapporterade värde')
print(previousvalues)

Senaste rapporterade värde
                     Reported percent Reported Date
Asset Name (Nasdaq)                                
BOL                              -0.1    2022-06-08
ELUX B                           -0.2    2022-06-08
ERIC B                           -0.7    2022-06-08
ESSITY B                         -0.7    2022-06-08
EVO                              -0.2    2022-06-08
HEXA B                           -0.5    2022-06-08
INVE B                           -0.6    2022-06-08
KINV B                           -0.7    2022-06-08
NDA SE                           -0.5    2022-06-08
SCA B                            -0.6    2022-06-08
SHB A                            -0.7    2022-06-08
SKA B                            -0.1    2022-06-08
SKF B                            -0.7    2022-06-08
SWED A                           -0.7    2022-06-08
SWMA                             -0.7    2022-06-08
TEL2 B                           -0.7    2022-06-08
TELIA                            -0.4

In [482]:
reportedValues = []

for index in OMX30List:

        name = index.symbol
        reportedDate = None
        previousValue = None
        lowerValue = None
        higherValue = None
        #Den bort kommenterade raden nedan kan användas för att testa koden
        postiontoReport = index.getPositiontoReport()*100#-0.3
        
        for row in previousvalues.iterrows():
            if(index.symbol == row[0]):
                reportedDate = row[1]['Reported Date']
                previousValue= row[1]['Reported percent']
                lowerValue = row[1]['Reported percent']
                higherValue = row[1]['Reported percent']-0.1
        
        tempReportedvalueObjet = reportValuesObjects(
            name,
            reportedDate,
            previousValue,
            lowerValue,
            higherValue,
            postiontoReport,
            calculatedValueRound,
            todaysDateWithLine)               
        reportedValues.append(tempReportedvalueObjet)

for row in reportedValues:
    
    calculatedValueRound = math.ceil(row.calculatedValue[0]*10)/10
    
    if(row.hasReportedValue()):
        print("name " + row.name)
        print("previousValue " + str(row.previousValue))
        print("lowerValue " + str(row.lowerValue))
        print("higherValue " + str(row.higherValue[0]))
        print("calculatedValue " + str(row.calculatedValue[0]))

        if not (row.lowerValue > row.calculatedValue[0] > row.higherValue[0]): 
            print("Värdet har ändrats "
                          + row.name
                          + '. Värdet är ' 
                          + str(row.calculatedValue[0])
                          + '/ '
                          + str(calculatedValueRound)
                          + ' och senaste rapporterade var ' 
                          + str(row.previousValue))
            
            new_row = {'Asset Name (Nasdaq)': row.name,
                       'Reported percent': row.calculatedValueRound,
                       'Reported Date': lastWeekdayDateWithLine }
            ReportedPositions_DataFrame = ReportedPositions_DataFrame.append(new_row, ignore_index=True)
            
    elif(row.calculatedValue[0] < -0.1):
        print("Nytt värde att rapportera för " 
              + row.name 
              + " är " 
              + str(row.calculatedValue[0])
              + '/ '
              + str(calculatedValueRound)          
             )
        new_row = {'Asset Name (Nasdaq)': row.name,
                   'Reported percent': row.calculatedValueRound,
                   'Reported Date': lastWeekdayDateWithLine }
        ReportedPositions_DataFrame = ReportedPositions_DataFrame.append(new_row, ignore_index=True) 


Senaste rapporterade värde
                     Reported percent Reported Date
Asset Name (Nasdaq)                                
BOL                              -0.1    2022-06-08
ELUX B                           -0.2    2022-06-08
ERIC B                           -0.7    2022-06-08
ESSITY B                         -0.7    2022-06-08
EVO                              -0.2    2022-06-08
HEXA B                           -0.5    2022-06-08
INVE B                           -0.6    2022-06-08
KINV B                           -0.7    2022-06-08
NDA SE                           -0.5    2022-06-08
SCA B                            -0.6    2022-06-08
SHB A                            -0.7    2022-06-08
SKA B                            -0.1    2022-06-08
SKF B                            -0.7    2022-06-08
SWED A                           -0.7    2022-06-08
SWMA                             -0.7    2022-06-08
TEL2 B                           -0.7    2022-06-08
TELIA                            -0.4

# Skapa historisk rapporterad position rapport

In [483]:


ReportedPositions_DataFrame.sort_values(by = ['Asset Name (Nasdaq)','Reported Date'])

writerReportedPositions_DataFrame = pd.ExcelWriter(reportedPostionFilename)
ReportedPositions_DataFrame.to_excel(writerReportedPositions_DataFrame, sheet_name='Arkiv', index=False)

for column in ReportedPositions_DataFrame:
    column_length = max(ReportedPositions_DataFrame[column].astype(str).map(len).max(), len(column))
    col_idx = ReportedPositions_DataFrame.columns.get_loc(column)
    writerReportedPositions_DataFrame.sheets['Arkiv'].set_column(col_idx, col_idx, column_length) 



In [484]:
writerReportedPositions_DataFrame.save()